In [1]:
import os

In [2]:
%pwd

'/Users/shrey_98/Desktop/MLOps_CNNClassifier/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/shrey_98/Desktop/MLOps_CNNClassifier'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from MLOPSCNNCLASSIFIER.constants import *
from MLOPSCNNCLASSIFIER.utils.common import read_yaml,create_directories

In [19]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
     

In [20]:
import os
import urllib.request as request
from zipfile import ZipFile 
import gdown
from MLOPSCNNCLASSIFIER import logger
from MLOPSCNNCLASSIFIER.utils.common import load_json
from src.MLOPSCNNCLASSIFIER.utils.common import get_size

In [21]:
class DataIngestion:

    def __init__(self,config:DataIngestionConfig):
        self.config = config
   

    def download_file(self)-> str:
        try:
                dataset_url=self.config.source_URL
                zip_download_dir=self.config.local_data_file
                os.makedirs("artifacts/data_ingestion",exist_ok=True)
                logger.info(f"Downloading data from {dataset_url} to {zip_download_dir}")

                file_id=dataset_url.split("/")[-2]
                prefix="https://drive.google.com/uc?/export=download&id="
                gdown.download(prefix+file_id,zip_download_dir)

                logger.info(f"Data downloaded from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
                raise e
    
    def extract_zip_file(self):
          # Extract the zip file into the data directory and returns None
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [23]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-03-18 16:56:30,505: INFO: common: YAML file: config/config.yaml loaded successfully]
[2024-03-18 16:56:30,507: INFO: common: YAML file: params.yaml loaded successfully]
[2024-03-18 16:56:30,508: INFO: common: Directory created at artifacts]
[2024-03-18 16:56:30,509: INFO: common: Directory created at artifacts/data_ingestion]
[2024-03-18 16:56:30,510: INFO: 2499461092: Downloading data from https://drive.google.com/file/d/1A0iqc9hefkzwZaK3llRWAGgdZMYN_5gV/view?usp=drive_link to artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1A0iqc9hefkzwZaK3llRWAGgdZMYN_5gV
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1A0iqc9hefkzwZaK3llRWAGgdZMYN_5gV&confirm=t&uuid=137b71bb-c26b-482e-8ef2-bb7f2dc96241
To: /Users/shrey_98/Desktop/MLOps_CNNClassifier/artifacts/data_ingestion/data.zip
100%|██████████| 943M/943M [00:20<00:00, 45.5MB/s] 

[2024-03-18 16:56:52,579: INFO: 2499461092: Data downloaded from https://drive.google.com/file/d/1A0iqc9hefkzwZaK3llRWAGgdZMYN_5gV/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
